In [1]:
"""
The intent of this notebook is dataset creation
for the MVP of our brainNN classifier.
"""
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tornado import gen
from tornado.ioloop import IOLoop
from aimetrics import metrics
import seaborn as sns
%matplotlib inline

In [2]:
# Retrieve the raw data for bst/drone
# NOTE: ASYNC
data = {}
@gen.coroutine
def get_data():
    data['drones'] = yield metrics.fetch_data("http://localhost:3002/", "bst", "drone", auth_username="bst", auth_password="bst")
    return data

IOLoop.instance().add_callback(get_data)

In [17]:
# separate and save the data into training, testing, and validation sets
from sklearn.cross_validation import StratifiedShuffleSplit
labels = data['drones']['classes']
X = pd.DataFrame(data['drones']['X'], columns=data['drones']['features'])[['rcs', 'range', 'speed']]
y = pd.DataFrame(data['drones']['y'], columns=labels)[['small drone', 'person']]
y['small_drone'] = y['small drone']
del y['small drone']
trn_val_ind, test_ind = list(StratifiedShuffleSplit(y['small_drone'], 1, 0.1))[0]
X_trn_val, X_test = X.iloc[trn_val_ind], X.iloc[test_ind]
y_trn_val, y_test = y.iloc[trn_val_ind], y.iloc[test_ind]
X_trn_val.to_csv('data/bnn-mvp/X_trn_val.csv')
y_trn_val.to_csv('data/bnn-mvp/y_trn_val.csv')
X_test.to_csv('data/bnn-mvp/X_test.csv')
y_test.to_csv('data/bnn-mvp/y_test.csv')